In [1]:
from qutip import *
from qutip.measurement import measure, measurement_statistics
import numpy as np
import matplotlib.pyplot as plt


In [217]:
N=65
#cavity operators
ac = destroy(N)
nb_op = ac.dag()*ac
ic = qeye(N)
#qubit operators
aq = destroy(2)
hq = (sigmax()+sigmaz())/np.sqrt(2)
iq = qeye(2)
g_proj = (iq+sigmaz())/2
e_proj = (iq-sigmaz())/2

# Experimental values

In [725]:
# frequencies
X_e0 = 2*np.pi*-93e3

X_f0 = 2*np.pi*-236e3

X_g0 = 0

#error times

ge_decay_time = 25e-6
fe_decay_time = 23e-6

g_dephasing_time = 81e-6
e_dephasing_time = 17e-6
f_dephasing_time = 12e-6

photon_loss_time = 1.07e-3

#transmon thermal population
n_th = 0.025


# Initial state

In [555]:
alpha = np.sqrt(2)
cat_state = (coherent(N,alpha)+coherent(N,-alpha)).unit()

# ge Parity Protocol

In [608]:
def ge_parity(cavity_state, collapse = []):
    
    H_int = X_e0*tensor(e_proj, nb_op)
    #H_parity = X_e*tensor(sigmaz(),nb_op)
    t = np.abs(np.pi/(X_e0))
    
    
    # prepare cat state and initial state
    psi0 = tensor((basis(2,0)+basis(2,1)).unit(), cavity_state)

    result = mesolve(H_int, psi0, np.linspace(0,t,200), c_ops = collapse)  
    
    psi = result.states[-1]
    
    if psi.type =='ket':
        psi = tensor(hq,ic)*psi
    elif psi.type == 'oper':
        psi = tensor(hq,ic)*psi*tensor(hq,ic).dag()
        
    return psi, result.states[-1]

In [609]:
ge_no_error, _ = ge_parity(cat_state)

In [610]:
ge_no_error.ptrace(0)

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[9.99999994e-01+0.00000000e+00j 4.58912264e-05-7.06603436e-06j]
 [4.58912264e-05+7.06603436e-06j 5.70372771e-09+0.00000000e+00j]]

In [611]:
fidelity(ge_no_error.ptrace(1), ket2dm(cat_state))

1.0000035159613805

## Relaxation error added

In [739]:
#collapse operator for relaxation
relax_error_rate_ge = np.sqrt(1/ge_decay_time)
relax_ge = tensor(relax_error_rate_ge*sigmap(),ic)

In [719]:
relax_error_rate_ge

40000.0

In [705]:
psi_ge_relax, re = ge_parity(cat_state, [relax_ge])

In [706]:
psi_ge_relax.ptrace(0)

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
Qobj data =
[[0.9490245 +0.00000000e+00j 0.09675096+1.47396479e-07j]
 [0.09675096-1.47396479e-07j 0.0509755 +0.00000000e+00j]]

In [707]:
fidelity(ket2dm(cat_state),psi_ge_relax.ptrace(1))

0.9698373137953185

## Dephasing error added

In [738]:
# collapse operator for dephasing
dephase_error_rate_g = np.sqrt(1/g_dephasing_time)
dephase_g = tensor(dephase_error_rate_g*basis(2,0).proj(),ic)

dephase_error_rate_e = np.sqrt(1/e_dephasing_time)
dephase_e = tensor(dephase_error_rate_e*basis(2,1).proj(),ic)

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[1. 0.]
 [0. 0.]]

In [721]:
psi_ge_dephase, _ = ge_parity(cat_state, [dephase_g, dephase_e])

In [724]:
psi_ge_dephase.ptrace(0)

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.5+0.00000000e+00j 0. -1.51076601e-09j]
 [0. +1.51076601e-09j 0.5+0.00000000e+00j]]

In [711]:
fidelity(ket2dm(cat_state),psi_ge_dephase.ptrace(1))

1.0000025189996693

## all errors added

In [740]:
photon_loss_rate = np.sqrt(1/photon_loss_time)
photon_loss = photon_loss_rate*tensor(iq,ac)

thermal_population_rate = np.sqrt(n_th/ge_decay_time)
thermal_population = thermal_population_rate*tensor(sigmam(),ic)

all_errors = [relax_ge, dephase_g, dephase_e, photon_loss, thermal_population]

In [741]:
thermal_population_rate

31.622776601683793

In [742]:
psi_ge_errors,_ = ge_parity(cat_state, all_errors)

In [743]:
psi_ge_errors.ptrace(0)

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
Qobj data =
[[0.86627578+0.j         0.09408812-0.00225156j]
 [0.09408812+0.00225156j 0.13372422+0.j        ]]

In [761]:
psi_ge_errors.ptrace(0).eigenstates()

(array([0.12182598+3.34093362e-15j, 0.87817402-3.34049994e-15j]),
 array([Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
        Qobj data =
        [[-0.12538807+0.00300058j]
         [ 0.99210323+0.j        ]]                                  ,
        Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
        Qobj data =
        [[0.99210323+0.j        ]
         [0.12538807+0.00300058j]]                                   ],
       dtype=object))

In [746]:
fidelity(ket2dm(cat_state),psi_ge_errors.ptrace(1))

0.9643455982154799

# gf Parity Protocol

In [750]:
#Three level ancilla operators
a3 = destroy(3)

half_pi_pulse_ge = Qobj([[1,1j,0],[1j,1,0],[0,0,np.sqrt(2)]]/np.sqrt(2))

pi_pulse_ef = Qobj([[1,0,0],[0,0,1j],[0,1j,0]])

iq3 = qeye(3)
g_proj3 = Qobj([[1,0,0],[0,0,0],[0,0,0]])
e_proj3 = Qobj([[0,0,0],[0,1,0],[0,0,0]])
f_proj3 = Qobj([[0,0,0],[0,0,0],[0,0,1]])
fe_proj = Qobj([[0,0,0],[0,0,1],[0,0,0]])

In [751]:
def gf_parity(cavity, collapse = []):
    chi = 1
    alpha = np.sqrt(2)
    nb_op = ac.dag()*ac

    H_int = X_e0*tensor(e_proj3, nb_op)+X_e0*tensor(f_proj3,nb_op)
    t = np.pi/np.abs(X_f0-X_g0)
    
    
    # prepare initial state
    qubit = basis(3,0)
    qubit = half_pi_pulse_ge*qubit
    qubit = pi_pulse_ef*qubit
    psi0 = tensor(qubit,cavity) 
    
    
    #result = mesolve(H_int, psi0, np.linspace(0,t,100))
    result = mesolve(H_int, psi0, np.linspace(0,t,100), c_ops = collapse)
    
    
    psi = result.states[-1]
    if psi.type == 'ket':
        psi = tensor(pi_pulse_ef,ic)*psi
        psi = tensor(half_pi_pulse_ge,ic)*psi
    elif psi.type == 'oper':
        psi = tensor(pi_pulse_ef,ic)*psi*tensor(pi_pulse_ef,ic).dag()
        psi = tensor(half_pi_pulse_ge,ic)*psi*tensor(half_pi_pulse_ge,ic).dag()
    
    return psi

In [755]:
#error operators
relax_error_rate_fe = np.sqrt(1/fe_decay_time)
relax_fe = relax_error_rate_fe*tensor(fe_proj,ic)

In [756]:
psi_gf = gf_parity(cat_state)

In [757]:
fidelity(psi_gf.ptrace(1),(coherent(N,alpha)+coherent(N,-alpha)).unit())

0.7217142752781164

In [758]:
psi_gf.ptrace(0)

Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = True
Qobj data =
[[0.44902992+0.00000000e+00j 0.08844075-2.81094047e-05j
  0.        +0.00000000e+00j]
 [0.08844075+2.81094047e-05j 0.55097008+0.00000000e+00j
  0.        +0.00000000e+00j]
 [0.        +0.00000000e+00j 0.        +0.00000000e+00j
  0.        +0.00000000e+00j]]